# Introduction

I am adekunle... i am making this project to follow up on the effects of cost of rent and how it affects the unemployment rate and lifes of average nigerian 

# Tools:
Tools i will be using are python,sql and tableau or power bi 


In [69]:
import httpx
from bs4 import BeautifulSoup
import pandas as pd
import time

## Getting the hosuing data i need first 

1.owner

2.price

3.Address/location

4.bedroom

5.bathroom

6.parking space


In [ ]:
# Base URL with a placeholder for page number
base_url = 'https://-----------.com/for-rent/lagos?page={}'

# Initialize lists to hold data
prices = []
addresses = []
owners = []
features = []

# Set the total number of pages to scrape (update this if needed)
total_pages = 1080  # Ensure this reflects the actual number of pages you want to scrape

# Loop through pages (1 to total_pages inclusive)
for page in range(1, total_pages + 1):
    url = base_url.format(page)
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"}

    max_retries = 3
    for attempt in range(max_retries):
        try:
            resp = httpx.get(url, headers=headers, timeout=10)
            if resp.status_code == 200:
                break
        except httpx.ReadTimeout:
            print(f"ReadTimeout occurred on attempt {attempt + 1}. Retrying...")
            time.sleep(2)  # Wait before retrying
    else:
        print(f"Failed to retrieve page {page} after {max_retries} attempts.")
        continue

    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # Extract data for each property listing
    titles = soup.find_all('div', class_='wp-block-content clearfix text-xs-left text-sm-left')
    
    if not titles:  # Check if there are no listings on the page
        print(f"No listings found on page {page}.")
        continue  # Skip to the next page if there are no listings

    for title in titles:
        # Extracting prices
        price = soup.find_all('span', class_='price')
        price = [i.text.replace("₦", "").strip() for i in price]
        prices.extend([i for i in price if i.replace(",", "").isdigit()])
        
        # Extracting addresses
        address = soup.find_all('address', class_='voffset-bottom-10')
        addresses.extend([i.text.strip() for i in address])
        
        # Extracting owners
        owner = soup.find_all('span', class_='marketed-by pull-right hidden-xs hidden-sm text-right')
        owners.extend([i.text.strip(" ") for i in owner])
        
        # Extracting features (bedrooms, bathrooms, etc.)
        feature = soup.find_all('div', class_='wp-block-footer')
        features.extend([i.text.split("s")[0:3] for i in feature])


    # Optional: Add a delay to avoid being blocked by the website
    time.sleep(5)  # Sleep for 2 seconds between requests


In [71]:
data_fr = pd.DataFrame()

In [72]:
data_fr['price'] = prices
data_fr['address'] = addresses
data_fr['features'] = features
data_fr['owner'] = owners



In [73]:
data_fr['features'] = data_fr['features'].astype(str)

In [74]:
data_fr[['Bedrooms', 'Bathrooms', 'Toilets']] = data_fr['features']\
    .str.replace(r'^\[\n*|\]$', '', regex=True)\
    .str.split(', ', expand=True)

In [75]:
data_fr['Bedrooms'] = data_fr['Bedrooms'].str.split().str[0]
data_fr['Bathrooms'] = data_fr['Bathrooms'].str.split().str[0]
data_fr['Toilets'] = data_fr['Toilets'].str.split().str[0]


In [76]:
data_fr['Bedrooms'] = data_fr['Bedrooms'].str.replace(r'\D', '', regex=True)  # Remove non-digit characters
data_fr['Bathrooms'] = data_fr['Bathrooms'].str.replace(r'\D', '', regex=True)
data_fr['Toilets'] = data_fr['Toilets'].str.replace(r'\D', '', regex=True)

In [79]:
data_fr

,price,address,features,owner,Bedrooms,Bathrooms,Toilets
0,"9,000,000","Off Freedom Way, Lekki Phase 1, Lekki, Lagos","['\n3 Bedroom', '3 Bathroom', '4 Toilet']",\nKam360 Degrees Realty\n\n 09052778215 \n,3,3,4
1,"15,000,000","Osapa, Osapa, Lekki, Lagos","['\n2 Bedroom', '2 Bathroom', '3 Toilet']",\nPresh Estate\n\n 08101406528 \n,2,2,3
2,"8,000,000","Orchid Road, Lekki, Lagos","['\n4 Bedroom', '4 Bathroom', '5 Toilet']",\nPresh Estate\n\n 08101406528 \n,4,4,5
3,"4,000,000","Ologolo Lekki Lagos, Ologolo, Lekki, Lagos","['\n2 Bedroom', '2 Bathroom', '3 Toilet']",\nS3 Global Height Limited\n\n 09069994668 \n,2,2,3
4,"4,500,000","Chevron Lekki Lagos, Lekki, Lagos","['\n2 Bedroom', '2 Bathroom', '3 Toilet']",\nS3 Global Height Limited\n\n 09069994668 \n,2,2,3
...,...,...,...,...,...,...,...
115096,"800,000","Lekki Phase 1, Lekki, Lagos",['\n\xa0 Save\n'],\nEffective-effektifs Homes And Realty\n\n 08...,0,None,None
115097,"24,000,000","Off Admiralty Way, Lekki Phase 1, Lekki, Lagos","['\n3 Bedroom', '3 Bathroom', '4 Toilet']",\nAdvanced Orients & Co\n\n 08033842143 \n,3,3,4
115098,"1,700,000","Lekki Phase 1, Lekki, Lagos",['\n1 Bathroom1 Toilet1 Parking Space Save\n'],\nGracealtar Homes\n\n +2348107230609 \n,1,None,None
115099,"700,000","Pedro, Gbagada, Lagos","['\n1 Bathroom1 Toilet5 Parking Space', '500 '...",\nWrix-way Properties Limited\n\n 09065544257 \n,1,500,


In [78]:
data_fr.to_csv('lagos housing prices raw')

In [80]:
pd.read_csv("lagos housing prices raw")

,Unnamed: 0,price,address,features,owner,Bedrooms,Bathrooms,Toilets
0,0,"9,000,000","Off Freedom Way, Lekki Phase 1, Lekki, Lagos","['3 Bedroom', '3 Bathroom', '4 Toilet']",\nKam360 Degrees Realty\n\n 09052778215 \n,3,3.0,4.0
1,1,"15,000,000","Osapa, Osapa, Lekki, Lagos","['2 Bedroom', '2 Bathroom', '3 Toilet']",\nPresh Estate\n\n 08101406528 \n,2,2.0,3.0
2,2,"8,000,000","Orchid Road, Lekki, Lagos","['4 Bedroom', '4 Bathroom', '5 Toilet']",\nPresh Estate\n\n 08101406528 \n,4,4.0,5.0
3,3,"4,000,000","Ologolo Lekki Lagos, Ologolo, Lekki, Lagos","['2 Bedroom', '2 Bathroom', '3 Toilet']",\nS3 Global Height Limited\n\n 09069994668 \n,2,2.0,3.0
4,4,"4,500,000","Chevron Lekki Lagos, Lekki, Lagos","['2 Bedroom', '2 Bathroom', '3 Toilet']",\nS3 Global Height Limited\n\n 09069994668 \n,2,2.0,3.0
...,...,...,...,...,...,...,...,...
115096,115096,"800,000","Lekki Phase 1, Lekki, Lagos",['\xa0 Save'],\nEffective-effektifs Homes And Realty\n\n 08...,0,NaN,NaN
115097,115097,"24,000,000","Off Admiralty Way, Lekki Phase 1, Lekki, Lagos","['3 Bedroom', '3 Bathroom', '4 Toilet']",\nAdvanced Orients & Co\n\n 08033842143 \n,3,3.0,4.0
115098,115098,"1,700,000","Lekki Phase 1, Lekki, Lagos",['1 Bathroom1 Toilet1 Parking Space Save'],\nGracealtar Homes\n\n +2348107230609 \n,1,NaN,NaN
115099,115099,"700,000","Pedro, Gbagada, Lagos","['1 Bathroom1 Toilet5 Parking Space', '500 ', ...",\nWrix-way Properties Limited\n\n 09065544257 \n,1,500.0,NaN
